# Vamos a utilizar SupaBase
- Una Base de datos en la nube que usa PostgreSQL
- Allí almacenaremos los datos recopilados
- La idea es poder tener la app funcionando desde cualquier lugar, que sea pública
- pip install python-dotenv psycopg2

Todo lo de aquí lo pasaremos a Funciones para facilitar subir datos en caso de ser necesario

In [2]:
import sys
sys.path.append("../")
import src.soporte_sql as sql
import pandas as pd
import ast

In [10]:
conexion = sql.conectar_bd()

Conectado a la base de datos


# Crear tablas
- Vamos a organizarlo por tablas

In [11]:
df = pd.read_csv("../datos/01 Spotify/01_followers_zara.csv")
df.sample()

,brand,username,user_id,playlists,playlist_ids,playlist_ids_limited,artistas
449,Zara,Lorena,teaaz8r3v1bui9dalkw9s8u1f,"{""DINNER MUSIC 2024/2025 🍷 New Year's chill co...","['2xPb74xnQZOh0FLPUw4ra6', '02SQYftwBgd64eeYXU...","['2xPb74xnQZOh0FLPUw4ra6', '02SQYftwBgd64eeYXU...",{'6lkrkVSqgzJZz2jguafSAy': 'Bossa Nova Covers'...


In [12]:
df2 = pd.read_csv("../datos/01 Spotify/01 Resumen Marcas/resumen_zara.csv")
df2.sample()

,brand,followers,unique_artists,artist_ranking,genres,genres_ranking
0,Zara,848,"{'3qm84nBOXUEQ2vnTfUTTFC': ""Guns N' Roses"", '6...","[('The Weeknd', 346), ('Dua Lipa', 298), ('Col...","['rock', 'glam metal', 'hard rock', 'classic r...","[('lo-fi', 1637), ('lo-fi beats', 1235), ('afr..."


# Tabla Brands
- Aquí almacenaremos los nombres de las marcas y su user id
### Crear Dataframe con formato tabla

In [13]:
dictio_marcas = {
    "zara" : "r6ivwuv0ebk346hhxo446pbfv",
    "primark" : "1u0ewgq4b77l1ttu6j4oyaylw",
    "nike" : "nikerunclub",
    "adidas" : "430wexvtgeypb6zdn45ge9c76",
    "h & m" : "hm",
    "pull & bear" : "pull_and_bear",
    "bershka" : "bershka",
    "stradivarius" : "stradivas",
    "mango" : "mango_official",
    "desigual" : "1119086036"
}
brands = pd.DataFrame({
    "brand" : list(dictio_marcas.keys()),
    "user_id" : list(dictio_marcas.values())
})

url_list = []
for id in brands["user_id"]:
    url_list.append(f"https://open.spotify.com/user/{id}")
brands["url"] = url_list
brands

,brand,user_id,url
0,zara,r6ivwuv0ebk346hhxo446pbfv,https://open.spotify.com/user/r6ivwuv0ebk346hh...
1,primark,1u0ewgq4b77l1ttu6j4oyaylw,https://open.spotify.com/user/1u0ewgq4b77l1ttu...
2,nike,nikerunclub,https://open.spotify.com/user/nikerunclub
3,adidas,430wexvtgeypb6zdn45ge9c76,https://open.spotify.com/user/430wexvtgeypb6zd...
4,h & m,hm,https://open.spotify.com/user/hm
5,pull & bear,pull_and_bear,https://open.spotify.com/user/pull_and_bear
6,bershka,bershka,https://open.spotify.com/user/bershka
7,stradivarius,stradivas,https://open.spotify.com/user/stradivas
8,mango,mango_official,https://open.spotify.com/user/mango_official
9,desigual,1119086036,https://open.spotify.com/user/1119086036


### Crear tabla y subir los contenidos del DF

In [14]:
conexion = sql.conectar_bd()
query = '''CREATE TABLE brands (
            id BIGINT PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
            name VARCHAR UNIQUE NOT NULL,
            user_id VARCHAR UNIQUE NOT NULL,
            url VARCHAR UNIQUE NOT NULL
            );'''
sql.modificar_bd(conexion,query)

Conectado a la base de datos
Se ha modificado correctamente la base de Datos


In [15]:
conexion = sql.conectar_bd()
query = '''INSERT INTO brands(name,user_id,url) VALUES (%s,%s,%s)'''
sql.insertar_muchos_datos(conexion,query,sql.generar_tupla(brands))

Conectado a la base de datos
Se han añadido los valores correctamente


In [16]:
brands.to_csv("../datos/02 Base de Datos/tabla_brands.csv",index=False)

# Tabla Followers
- Aquí estará toda la información de los seguidores, y de que marca provienen

In [35]:
zara = pd.read_csv('../datos/01 Spotify/01_followers_zara.csv')
primark = pd.read_csv("../datos/01 Spotify/02_followers_primark.csv",index_col=0)
nike = pd.read_csv("../datos/01 Spotify/03_followers_nike.csv",index_col=0)
adidas = pd.read_csv("../datos/01 Spotify/04_followers_adidas.csv",index_col=0)
hm = pd.read_csv("../datos/01 Spotify/05_followers_hm.csv",index_col=0)
pullbear = pd.read_csv("../datos/01 Spotify/06_followers_pull_and_bear.csv",index_col=0)
bershka = pd.read_csv("../datos/01 Spotify/07_followers_bershka.csv",index_col=0)
stradivarius = pd.read_csv("../datos/01 Spotify/08_followers_stradivarius.csv",index_col=0)
mango = pd.read_csv("../datos/01 Spotify/09_followers_mango.csv",index_col=0)
desigual = pd.read_csv("../datos/01 Spotify/10_followers_desigual.csv",index_col=0)

In [36]:
zara = zara[["brand","username","user_id"]]
primark = primark[["brand","username","user_id"]]
nike = nike[["brand","username","user_id"]]
adidas = adidas[["brand","username","user_id"]]
hm = hm[["brand","username","user_id"]]
pullbear = pullbear[["brand","username","user_id"]]
bershka = bershka[["brand","username","user_id"]]
stradivarius = stradivarius[["brand","username","user_id"]]
mango = mango[["brand","username","user_id"]]
desigual = desigual[["brand","username","user_id"]]
followers = pd.concat([zara,primark,nike,adidas,hm,pullbear,bershka,stradivarius,mango,desigual])
followers = followers[["username","user_id","brand"]]
followers.sample()

,username,user_id,brand
227,Dinni⁎˳.:*♡☆,eyn8r2s414eh4uq369esvxg8s,H&M


### Preparar tabla
- Pasamos "brand" al id que tenemos en la tabla "brands" para llamara a la clave foránea

In [37]:
dictio_brand = {
    "Zara": 1,
    "Primark": 2,
    "Nike": 3,
    "Adidas": 4,
    "H&M": 5,
    "Pull & Bear": 6,
    "bershka": 7,
    "Stradivarius": 8,
    "Mango": 9,
    "Desigual": 10
}
followers["brand"] = followers["brand"].map(dictio_brand)
followers.sample()

,username,user_id,brand
236,cicrrr,cicrrr,6


### Crear tabla y subir los contenidos del DF

In [38]:
conexion = sql.conectar_bd()
query = ''' CREATE TABLE followers (
            id BIGINT PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
            username VARCHAR NOT NULL,
            user_id VARCHAR NOT NULL,
            brand_id BIGINT REFERENCES brands(id) ON DELETE CASCADE
            );'''
sql.modificar_bd(conexion,query)

Conectado a la base de datos
Se ha modificado correctamente la base de Datos


In [ ]:
conexion = sql.conectar_bd()
query = '''INSERT INTO followers(username,user_id,brand_id) VALUES (%s,%s,%s)'''
sql.insertar_muchos_datos(conexion,query,sql.generar_tupla(followers))

Conectado a la base de datos
Se han añadido los valores correctamente


In [41]:
followers.to_csv("../datos/02 Base de Datos/02_tabla_followers.csv",index=False)

# Tabla Playlists
- Esta tabla tendra cada playlist con su id, su nombre y la marca de donde procede
- además estarán referenciadas por brand y user_id

In [101]:
zara = pd.read_csv('../datos/01 Spotify/01_followers_zara.csv')
primark = pd.read_csv("../datos/01 Spotify/02_followers_primark.csv",index_col=0)
nike = pd.read_csv("../datos/01 Spotify/03_followers_nike.csv",index_col=0)
adidas = pd.read_csv("../datos/01 Spotify/04_followers_adidas.csv",index_col=0)
hm = pd.read_csv("../datos/01 Spotify/05_followers_hm.csv",index_col=0)
pullbear = pd.read_csv("../datos/01 Spotify/06_followers_pull_and_bear.csv",index_col=0)
bershka = pd.read_csv("../datos/01 Spotify/07_followers_bershka.csv",index_col=0)
stradivarius = pd.read_csv("../datos/01 Spotify/08_followers_stradivarius.csv",index_col=0)
mango = pd.read_csv("../datos/01 Spotify/09_followers_mango.csv",index_col=0)
desigual = pd.read_csv("../datos/01 Spotify/10_followers_desigual.csv",index_col=0)
zara = zara[["brand","username","user_id","playlists"]]
playlists = pd.concat([zara,primark,nike,adidas,hm,pullbear,bershka,stradivarius,mango,desigual])

### Obtenemos el id de los usuarios haciendo una consulta a la bd

In [102]:
conexion = sql.conectar_bd()
query = '''SELECT * FROM followers'''
df_users = sql.consulta_sql(conexion,query)
df_users = df_users[["id","user_id","username"]]
df_users.sample()

Conectado a la base de datos


,id,user_id,username
3758,3759,n5pntaz8mzfbyj6pcqd10sk4w,D4bren


### Realizamos merge y convertimos a diccionario "playlists"

In [103]:
unir = pd.merge(left=df_users,right=playlists,on="user_id")
playlists = unir[["id","playlists"]]
playlists["playlists"] = playlists["playlists"].apply(ast.literal_eval)
playlists.sample()

,id,playlists
6890,5643,"{'Navidad': '6VEY42tqjOurvtSuc4F4Ay', 'Bonus':..."


### Creamos un Dataframe que:
- tendra el id del usuario a quien pertenece
- nombre de la playlist
- id de la playlist
- Uno por fila

In [104]:
id_list = []
playlist_names = []
playlist_ids = []

for index, fila in playlists.iterrows():
    user_id = fila["id"]
    playlist = fila["playlists"]

    for playlist_name, playlist_id in playlist.items():
        id_list.append(user_id)
        playlist_names.append(playlist_name)
        playlist_ids.append(playlist_id)

playlists =  pd.DataFrame({
    "id" : id_list,
    "playlist_name" : playlist_names,
    "playlist_id" : playlist_ids
})
playlists = playlists[["playlist_name","playlist_id","id"]]
playlists.to_csv("../datos/02 Base de Datos/03_tabla_playlists.csv", index=False)
playlists.sample()

,playlist_name,playlist_id,id
55835,💐🍂,7Ez2es9sveVEW2ywOwWfVJ,1947


### Crear Tabla

In [105]:
conexion = sql.conectar_bd()
query = ''' CREATE TABLE playlists (
            id BIGINT PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
            playlist_name VARCHAR NOT NULL,
            playlist_id VARCHAR NOT NULL,
            follower_id BIGINT REFERENCES followers(id) ON DELETE CASCADE
            );'''
sql.modificar_bd(conexion,query)

Conectado a la base de datos
Se ha modificado correctamente la base de Datos


In [106]:
conexion = sql.conectar_bd()
query = '''INSERT INTO playlists(playlist_name,playlist_id,follower_id) VALUES (%s,%s,%s)'''
sql.insertar_muchos_datos(conexion,query,sql.generar_tupla(playlists))

Conectado a la base de datos
Se han añadido los valores correctamente


# Tabla Reduced Playlists
- Esta tabla busca tener solo los ids de las playlists que finalmente se usaron para sacar los artistas y géneros.
- De esta forma guardamos todas las playlists en un lado y por otro lado, mostramos las que finalmente usamos.

In [107]:
# Cargar los datasets de las marcas
zara = pd.read_csv('../datos/01 Spotify/01_followers_zara.csv')
primark = pd.read_csv("../datos/01 Spotify/02_followers_primark.csv", index_col=0)
nike = pd.read_csv("../datos/01 Spotify/03_followers_nike.csv", index_col=0)
adidas = pd.read_csv("../datos/01 Spotify/04_followers_adidas.csv", index_col=0)
hm = pd.read_csv("../datos/01 Spotify/05_followers_hm.csv", index_col=0)
pullbear = pd.read_csv("../datos/01 Spotify/06_followers_pull_and_bear.csv", index_col=0)
bershka = pd.read_csv("../datos/01 Spotify/07_followers_bershka.csv", index_col=0)
stradivarius = pd.read_csv("../datos/01 Spotify/08_followers_stradivarius.csv", index_col=0)
mango = pd.read_csv("../datos/01 Spotify/09_followers_mango.csv", index_col=0)
desigual = pd.read_csv("../datos/01 Spotify/10_followers_desigual.csv", index_col=0)

# Seleccionar columnas clave de Zara
zara = zara[["brand", "username", "user_id", "playlists"]]

# Concatenar todos los datasets en un único DataFrame
playlists = pd.concat([zara, primark, nike, adidas, hm, pullbear, bershka, stradivarius, mango, desigual])
# Convertir las cadenas de texto de la columna "playlists" a diccionarios reales
playlists["playlists"] = playlists["playlists"].apply(ast.literal_eval)

### Reducir a 10 elementos por diccionario
- Será las playlists que usemos para sacar los géneros

In [108]:
# Lista para almacenar los diccionarios reducidos
reduced_playlists_list = []

# Iterar sobre cada diccionario en la columna "playlists"
for dict_item in playlists["playlists"]:
    reduced_dict = {}  # Diccionario reducido
    count = 0  # Contador para elementos procesados

    # Iterar sobre los elementos del diccionario y limitar a 10
    for key, value in dict_item.items():
        if count < 10:
            reduced_dict[key] = value
            count += 1
        else:
            break
    
    # Agregar el diccionario reducido a la lista
    reduced_playlists_list.append(reduced_dict)

# Crear la nueva columna "reduced_playlists" en el DataFrame
playlists["reduced_playlists"] = reduced_playlists_list

In [109]:
playlists = playlists[["user_id","reduced_playlists"]]
playlists.sample()

,user_id,reduced_playlists
542,lele029,"{'Lélé': '5ud4RQdsz5w61ED0BdOPbu', 'Enfants': ..."


### Obtenemos el id de los usuarios haciendo una consulta a la bd

In [110]:
conexion = sql.conectar_bd()
query = '''SELECT * FROM followers'''
df_users = sql.consulta_sql(conexion,query)
df_users = df_users[["id","user_id","username"]]
df_users.sample()

Conectado a la base de datos


,id,user_id,username
7005,7006,researchingtheweb,researchingtheweb


### Realizamos merge y convertimos a diccionario "playlists"

In [111]:
unir = pd.merge(left=df_users,right=playlists,on="user_id")
playlists = unir[["id","reduced_playlists"]]
playlists.sample()

,id,reduced_playlists
4975,4079,"{'Spinning L': '6Kh64jKMmR2nirHMnVCDT5', 'Lẞ^2..."


In [112]:
id_list = []
playlist_names = []
playlist_ids = []

for index, fila in playlists.iterrows():
    user_id = fila["id"]
    playlist = fila["reduced_playlists"]

    for playlist_name, playlist_id in playlist.items():
        id_list.append(user_id)
        playlist_names.append(playlist_name)
        playlist_ids.append(playlist_id)

playlists =  pd.DataFrame({
    "id" : id_list,
    "playlist_name" : playlist_names,
    "playlist_id" : playlist_ids
})
playlists = playlists[["playlist_name","playlist_id","id"]]
playlists.to_csv("../datos/02 Base de Datos/04_tabla_reduced_playlists.csv", index=False)
playlists.sample()

,playlist_name,playlist_id,id
17505,Musikquiz Clara,1nQcSQBv39Y109HgekQ7ye,2060


### Crear Tabla

In [113]:
conexion = sql.conectar_bd()
query = ''' CREATE TABLE reduced_playlists (
            id BIGINT PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
            playlist_name VARCHAR NOT NULL,
            playlist_id VARCHAR NOT NULL,
            follower_id BIGINT REFERENCES followers(id) ON DELETE CASCADE
        );'''
sql.modificar_bd(conexion,query)

Conectado a la base de datos
Se ha modificado correctamente la base de Datos


In [114]:
conexion = sql.conectar_bd()
query = '''INSERT INTO reduced_playlists(playlist_name,playlist_id,follower_id) VALUES (%s,%s,%s)'''
sql.insertar_muchos_datos(conexion,query,sql.generar_tupla(playlists))

Conectado a la base de datos
Se han añadido los valores correctamente


# Tabla Artistas
- Tiene el nombre del Artista
- Conectado a la playlist de la que viene
- Conectado a la marca del que proviene

In [20]:
artistas = pd.read_csv("../datos/01 Spotify/01 Resumen Marcas/resumen_zara.csv")
artistas

,brand,followers,unique_artists,artist_ranking,genres,genres_ranking
0,Zara,848,"{'3qm84nBOXUEQ2vnTfUTTFC': ""Guns N' Roses"", '6...","[('The Weeknd', 346), ('Dua Lipa', 298), ('Col...","['rock', 'glam metal', 'hard rock', 'classic r...","[('lo-fi', 1637), ('lo-fi beats', 1235), ('afr..."


### Nos quedamos con brand y unique_artists

In [21]:
artistas = artistas[["brand","unique_artists"]]
# Convertimos unique_artists a diccionario
artistas["unique_artists"] = artistas["unique_artists"].apply(ast.literal_eval)
artistas

,brand,unique_artists
0,Zara,"{'3qm84nBOXUEQ2vnTfUTTFC': 'Guns N' Roses', '6..."


### Crear tabla separando el diccionario

In [22]:
lista_brand = []
nombres_artistas = []
ids_artistas = []

for dictio in artistas["unique_artists"]:
    for artist_id, artist_name in dictio.items():
        lista_brand.append(1) # El id de zara en la bd es 1
        nombres_artistas.append(artist_name)
        ids_artistas.append(artist_id)
artistas = pd.DataFrame({
    "artist_name" : nombres_artistas,
    "artist_id" : ids_artistas,
    "brand_id" : lista_brand
})
artistas.to_csv("../datos/02 Base de Datos/05_tabla_artistas_zara.csv",index=False)
artistas.head()

,artist_name,artist_id,brand_id
0,Guns N' Roses,3qm84nBOXUEQ2vnTfUTTFC,1
1,Nirvana,6olE6TJLqED3rqDCT0FyPh,1
2,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,1
3,Led Zeppelin,36QJpDe2go2KgaRleHCDTp,1
4,The Rolling Stones,22bE4uQ6baNwSHPVcDxLCe,1


### Quitar Nulos

In [44]:
artistas.dropna(inplace=True)

### Crear Tabla
- Empezamos a usar text porque es mucho mejor que varchar para nuestro uso

In [46]:
conexion = sql.conectar_bd()
query = ''' create table artists (
            id bigint primary key generated always as identity,
            artist_name text not null,
            artist_id text not null,
            brand_id bigint references brands (id) on delete cascade
            );'''
sql.modificar_bd(conexion,query)

Conectado a la base de datos
Se ha modificado correctamente la base de Datos


### Insertamos Datos

In [47]:
conexion = sql.conectar_bd()
query = '''INSERT INTO artists(artist_name,artist_id,brand_id) VALUES (%s,%s,%s)'''
sql.insertar_muchos_datos(conexion,query,sql.generar_tupla(artistas))

Conectado a la base de datos
Se han añadido los valores correctamente


# Tabla Ranking Artistas
- Tiene el nombre del Artista
- Y el número de veces que aparece un artista
- Conectado a la marca del que proviene

In [79]:
def subir_ranking_artistas(ranking_df,brand_id = 0, ruta_csv = "../datos/02 Base de Datos/tempsave_ranking.csv"):
    conexion = sql.conectar_bd()
    query = ''' SELECT * FROM brands'''
    df = sql.consulta_sql(conexion, query)
    df = df[["id","name"]]
    check = df["id"].tolist()
    if brand_id not in check:
        print("El id de la marca no existe")
        print("Los ids de las marcas son:")
        return display(df)

    ranking_art = ranking_df[["brand","artist_ranking"]]
    # Convertimos unique_artists a lista
    ranking_art["artist_ranking"] = ranking_art["artist_ranking"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
    
    lista_brands = []
    lista_artistas = []
    lista_conteos = []
    for tuplas in ranking_art["artist_ranking"]:
        for artista, conteo in tuplas:
            lista_artistas.append(artista)
            lista_conteos.append(conteo)
            lista_brands.append(brand_id) # el id de la marca

    ranking_art = pd.DataFrame({
        "artist_name" : lista_artistas,
        "number_of_appearances" : lista_conteos,
        "brand_id" : lista_brands
    })
    # Guardar en CSV el archivo
    ranking_art.to_csv(ruta_csv,index=False)
    conexion = sql.conectar_bd()
    query = '''INSERT INTO artists_ranking(artist_name,number_of_appearances,brand_id) VALUES (%s,%s,%s)'''
    sql.insertar_muchos_datos(conexion,query,sql.generar_tupla(ranking_art))

In [65]:
ranking_art = pd.read_csv("../datos/01 Spotify/01 Resumen Marcas/resumen_zara.csv")
ranking_art = ranking_art[["brand","artist_ranking"]]
ranking_art["artist_ranking"] = ranking_art["artist_ranking"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
ranking_art

,brand,artist_ranking
0,Zara,"[(The Weeknd, 346), (Dua Lipa, 298), (Coldplay..."


In [66]:
lista_brands = []
lista_artistas = []
lista_conteos = []
for tuplas in ranking_art["artist_ranking"]:
    for artista, conteo in tuplas:
        lista_artistas.append(artista)
        lista_conteos.append(conteo)
        lista_brands.append(1) # 1 es zara en la base de datos

ranking_art = pd.DataFrame({
    "artist_name" : lista_artistas,
    "number_of_appearances" : lista_conteos,
    "brand_id" : lista_brands
})

ranking_art.head()

,artist_name,number_of_appearances,brand_id
0,The Weeknd,346,1
1,Dua Lipa,298,1
2,Coldplay,282,1
3,Rihanna,279,1
4,Billie Eilish,276,1


In [64]:
conexion = sql.conectar_bd()
query = ''' create table artists_ranking (
            id bigint primary key generated always as identity,
            artist_name text not null,
            number_of_appearances int not null,
            brand_id bigint references brands (id) on delete cascade
          );'''
sql.modificar_bd(conexion,query)

Conectado a la base de datos
Se ha modificado correctamente la base de Datos


In [67]:
conexion = sql.conectar_bd()
query = '''INSERT INTO artists_ranking(artist_name,number_of_appearances,brand_id) VALUES (%s,%s,%s)'''
sql.insertar_muchos_datos(conexion,query,sql.generar_tupla(ranking_art))

Conectado a la base de datos
Se han añadido los valores correctamente


In [68]:
ranking_art.to_csv("../datos/02 Base de Datos/06_ranking_artistas_zara.csv",index=False)